THIS IS THE MAIN FUNCTION INCLUDES: EXTRACTING STREAMING DATA FROM TWITTER, PRE-PROCESSING AND STORES INTO MYSQL.

In [1]:
#SETTING UP

import credentials              #IMPORTING CREDENTIALS.PY - IMPORTS API/ACCESS_TOKEN KEYS
import settings                 #IMPORT SETTINGS
import re
import tweepy
import pandas as pd
from textblob import TextBlob
import mysql.connector
import demoji

In [2]:
# STORE DATA IN MYSQL, CONNECTING TO SQLDATABASE
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd=credentials.MYSQLPASSWORD,
    database="twitterdb",
    auth_plugin='mysql_native_password',
    charset = 'utf8'
)

if mydb.is_connected():
#CHECK TO SEE IF TABLE EXISTS. IF NOT, CREATES ONE.
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()

In [3]:
#FUNCTIONS

#PRE-PROCESSING
def clean_tweet(tweet):
    post=tweet.text
    try:
        post=re.sub("\n", "", post)   #REMOVE "\n" IN TWEETS, \n SEPERATES LINES VIA NEW LINE CHARACTER, REMOVES CLUTER
        post=re.sub(r'https\S+','',post) #REMOVE LINKS
        post=re.sub(r'www\.\S+', '', post) #REMOVE LINKS
        post=re.sub(r'@\S+\s?', '', post) #REMOVE MENTIONS
        post=demoji.replace(post, '') #REMOVE EMOJIS
        post=re.sub(r'ID\S+\s?','', post) #REMOVE ID/REFERALS IN TWEETS
        post=re.sub(r'0x\S+\s?','', post) #REMOVE ETH WALLET ADDRESSES IN TWEETS
        
    except AttributeError:
        post = tweet
    #TWITTER NEW TEXT LIMITS FOR TWITTER BLUE USERS INCLUDES TWEET LENGTH OF UPTO 4,000 CHARACTERS. WILL STORE TWEETS UPTO 255 CHRACTERS.
    return post.strip()[:255]


#SQL PUSH TO TABLES, FUNCTION INSERTS DATA INTO TABLE USING DATA IN THE 'STRUCT' DICTIONARY
def push_results_to_tables(table_name, struct, conn):
    cursor = conn.cursor()
    insert_SQL = f"""INSERT INTO {table_name}
                ({', '.join(map(str, struct))})
                VALUES('{"','".join(map(str,struct.values()))}');
                """
    cursor.execute(insert_SQL)
    cursor.commit()

In [4]:
class MyStream(tweepy.StreamingClient):

    # DISPLAYS "CONNECTED" ONCE STREAM IS CONNECTED
    def on_connect(self):        
        print("Connected") 

    def on_tweet(self,tweet):   
        
    #EXTRACTING ATTRIBUTES FROM TWEETS
        id_str = tweet.id
        created_at = tweet.created_at
        text = clean_tweet(tweet) # PRE-PROCESSING
        user_id = tweet.author_id
        ref_id=tweet.referenced_tweets
        lang=tweet.lang
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        struct={'id_str':id_str,
                'created_at':tweet.created_at,
                'text':text,
                'polarity':polarity,
                'subjectivity':subjectivity}
        print(struct)

        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity) VALUES (%s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, )
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()

## TWITTER HAS RATE LIMITS, STOP DATA SCARPING AFTER THRESHOLD.
    def on_error(self,status_code):
        if status_code == 420:
            return False
        print(status_code)
         

In [5]:
stream = MyStream(bearer_token=credentials.BEARER_TOKEN,wait_on_rate_limit=True)

# CLEARS RULESET BEFORE STREAMING DATA
for rule in stream.get_rules().data:
        stream.delete_rules(rule.id)

# ADDING RULES TO RULESET TO STREAM SPECIFIC DATA - ONLY ORIGINAL ENGLISH TWEETS WITH NO ATACHMENTS AND FILTERED QUERY ARE STORED.

stream.add_rules(tweepy.StreamRule('#ETH -is:retweet -is:quote -is:reply lang:en -has:media -faucet -bot -bots -nfts -nft -#nfts -#nft -"my TL" -"Public Sale" -"Price Update:" -giveaway -gwei -"purchased for" -"transferred from" -airdrop -claim -$ARB -address -"check out" -"earn over" -#sportsbet -"Market Updates" -"Alpha Trade" -#USDC -"live:" -signal -"percent in the last" -#alert -Arbitrum -"pay your bills" -link -#whale -"Sold at" -#ENS -Shakepay'))
#START STREAM
stream.filter(expansions=["author_id",],tweet_fields=["created_at","referenced_tweets","lang","attachments"]) 





Connected
{'id_str': 1638218668545921025, 'created_at': datetime.datetime(2023, 3, 21, 16, 39, 1, tzinfo=datetime.timezone.utc), 'text': '#ETHereum #ETH impulse bar 1823.12 - 1831.35$ETH Bar close 1827.33#Crypto short term volatility detection', 'polarity': 0.0, 'subjectivity': 0.3}
{'id_str': 1638218890164699136, 'created_at': datetime.datetime(2023, 3, 21, 16, 39, 54, tzinfo=datetime.timezone.utc), 'text': 'Crypto News: These 3 Cointelegraph Markets Pro alerts generated a cumulative profit of over 100% ;  ----- #BTC #Bitcoin #ETH #Cryptocurrency #cryptoupdate #Crash #Bullish #Bearish #Ethereum #Binance #BNB', 'polarity': 0.0, 'subjectivity': 0.0}
{'id_str': 1638219009316335617, 'created_at': datetime.datetime(2023, 3, 21, 16, 40, 23, tzinfo=datetime.timezone.utc), 'text': 'FCA confiscates ATMs in the UK Read More-  #crypto #cryptonews #cryptolatest #cryptoupdates #bitcoinnews #btc #eth #sol #xrp #bnb', 'polarity': 0.5, 'subjectivity': 0.5}
{'id_str': 1638219020133597184, 'created_at'

KeyboardInterrupt: 

In [6]:

stream.get_rules()
print(stream.get_rules())

Response(data=[StreamRule(value='#ETH -is:retweet -is:quote -is:reply lang:en -has:media -faucet -bot -bots -nfts -nft -#nfts -#nft -"my TL" -"Public Sale" -"Price Update:" -giveaway -gwei -"purchased for" -"transferred from" -airdrop -claim -$ARB -address -"check out" -"earn over"', tag=None, id='1638013632477818881')], includes={}, errors=[], meta={'sent': '2023-03-21T03:04:42.910Z', 'result_count': 1})
